<a href="https://colab.research.google.com/github/Veerabhadra-YU/Natural-Language-Processing-NLP-Course/blob/main/Assignment_4_LSTM_part_III_Veerabhadra_Rao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part III
Using the previous two tutorials, please answer the following using an encorder-decoder approach and an LSTM compared approach.

Please create a transformer-based classifier for English name classification into male or female.

There are several datasets for name for male or female classification. In subseuqent iterations, this could be expanded to included more classifications.

Below is the source from NLTK, which only has male and female available but could be used for the purposes of this assignment.

```
names = nltk.corpus.names
names.fileids()
['female.txt', 'male.txt']
male_names = names.words('male.txt')
female_names = names.words('female.txt')
[w for w in male_names if w in female_names]
['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis',
'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel',
'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', ...]
```

In [ ]:
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
None
### END CODE HERE ###

In [ ]:
!pip install tensorflow_datasets
!pip install -U tensorflow-text

In [ ]:
pip install tensorflow

# **LSTM-based Approach**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

# Define a simple LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Prepare data
all_names = male_names + female_names
labels = [0] * len(male_names) + [1] * len(female_names)

# Encode names
char_to_idx = {char: idx for idx, char in enumerate('abcdefghijklmnopqrstuvwxyz')}
max_len = max(len(name) for name in all_names)
def encode_name(name):
    name = name.lower()
    encoded = np.zeros((max_len, 26))
    for i, char in enumerate(name):
        encoded[i][char_to_idx[char]] = 1
    return encoded

encoded_names = [encode_name(name) for name in all_names]

X_train, X_val, y_train, y_val = train_test_split(encoded_names, labels, test_size=0.2, random_state=42)

# Convert data to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

# Create data loaders
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False)

# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(input_size=26, hidden_size=128, num_layers=2, num_classes=2).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = correct_preds / total_preds

    # Validation
    model.eval()
    val_running_loss = 0.0
    val_correct_preds = 0
    val_total_preds = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            val_loss = criterion(outputs, labels)

            val_running_loss += val_loss.item()

            _, predicted = torch.max(outputs, 1)
            val_correct_preds += (predicted == labels).sum().item()
            val_total_preds += labels.size(0)

    val_loss = val_running_loss / len(val_loader)
    val_acc = val_correct_preds / val_total_preds

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')


<ipython-input-40-0b0a309269b6>:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  X_train = torch.tensor(X_train, dtype=torch.float32)


Epoch 1/10, Train Loss: 0.6950, Train Acc: 0.4103, Val Loss: 0.6971, Val Acc: 0.3000
Epoch 2/10, Train Loss: 0.6871, Train Acc: 0.5641, Val Loss: 0.7161, Val Acc: 0.3000
Epoch 3/10, Train Loss: 0.6870, Train Acc: 0.5641, Val Loss: 0.7388, Val Acc: 0.3000
Epoch 4/10, Train Loss: 0.6825, Train Acc: 0.5641, Val Loss: 0.7457, Val Acc: 0.3000
Epoch 5/10, Train Loss: 0.6764, Train Acc: 0.5641, Val Loss: 0.7330, Val Acc: 0.3000
Epoch 6/10, Train Loss: 0.6735, Train Acc: 0.5641, Val Loss: 0.7256, Val Acc: 0.3000
Epoch 7/10, Train Loss: 0.6562, Train Acc: 0.5641, Val Loss: 0.7045, Val Acc: 0.3000
Epoch 8/10, Train Loss: 0.6181, Train Acc: 0.5897, Val Loss: 0.6226, Val Acc: 0.4000
Epoch 9/10, Train Loss: 0.5337, Train Acc: 0.6923, Val Loss: 0.4835, Val Acc: 0.6000
Epoch 10/10, Train Loss: 0.4386, Train Acc: 0.8462, Val Loss: 0.3976, Val Acc: 0.8000


In [ ]:
# Define a function to predict classes for a set of names
def predict_classes(model, loader):
    model.eval()
    predictions = []
    true_labels = []

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels

# Test the model on the test set
test_predictions, test_true_labels = predict_classes(model, val_loader)

# Calculate test accuracy
test_correct_preds = sum(p == t for p, t in zip(test_predictions, test_true_labels))
test_accuracy = test_correct_preds / len(test_true_labels)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Map predicted labels to class names
class_names = ['male', 'female']
predicted_classes = [class_names[label] for label in test_predictions]

# Print each name along with its predicted class
for name, predicted_class in zip(test_names, predicted_classes):
    print(f'{name}: {predicted_class}')


Test Accuracy: 0.8000
Allyn: female
Angelia: female
Angelina: female
Angela: female
Angel: female
Alina: female
Alicia: female
Alice: female
Alma: female
Ariel: male


In [1]:
%%shell
jupyter nbconvert --to html /content/Assignment_4_LSTM_part_III_Veerabhadra_Rao.ipynb

[NbConvertApp] Converting notebook /content/Assignment_4_LSTM_part_III_Veerabhadra_Rao.ipynb to html
[NbConvertApp] Writing 626774 bytes to /content/Assignment_4_LSTM_part_III_Veerabhadra_Rao.html


# References
1. https://arxiv.org/pdf/2102.03692.pdf
2. https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/exercise/13-attention.html
3. https://towardsdatascience.com/deep-learning-gender-from-name-lstm-recurrent-neural-networks-448d64553044
4. https://www.nltk.org/book/ch02.html#sec-lexical-resources